In [ ]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
label_decoder = {
    "en": 0,
    "es": 1,
    "de": 2
}

In [ ]:
def one_hot_encode(i):
    label_one_hot = [0, 0, 0]
    label_one_hot[i] = 1
    return label_one_hot

In [ ]:
def create_database(path):
    image_directory = os.listdir(path)
    image_labels = image_directory
    image_directory = [path + i for i in image_directory]
    images = []
    labels = []
    for i in image_directory:
        flag_images_dir = [(i+'/'+j) for j in os.listdir(i)]
        flag_labels = [(label_decoder[i[-2:]]) for j in os.listdir(i)]
        flag_labels = [one_hot_encode(i) for i in flag_labels]
        labels = labels + flag_labels
        images = images + flag_images_dir
    return images, labels

In [ ]:
images_train, labels_train = create_database('drive/My Drive/train_spectrogram_images/')
images_test, labels_test = create_database('drive/My Drive/test_spectrogram_images/')

In [ ]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [432, 432])
    image /= 255.0
    return image
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)
def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), label

In [ ]:
BATCH_SIZE = 16
AUTOTUNE = 1
image_count = len(images_train)

In [ ]:
def get_iterator(images, labels):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    image_label_ds = dataset.map(load_and_preprocess_from_path_label)
    dataset = image_label_ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
    dataset = dataset.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
    iterator = dataset.make_one_shot_iterator()
    return iterator

iterator_train = get_iterator(images_train, labels_train)
iterator_test = get_iterator(images_test, labels_test)

In [ ]:
n_classes = 3

In [ ]:
def conv2d(x, w):
    return tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME')
def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [ ]:
def convolutional_neural_network(x):
    weights = {
        'w_conv1':tf.Variable(tf.random_normal([5,5,3,32])),
        'w_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
        'w_conv3':tf.Variable(tf.random_normal([5,5,64,128])),
        'w_fc':tf.Variable(tf.random_normal([54*54*128,1000])),
        'out':tf.Variable(tf.random_normal([1000, n_classes]))
    }

    biases = {
        'b_conv1':tf.Variable(tf.random_normal([32])),
        'b_conv2':tf.Variable(tf.random_normal([64])),
        'b_conv3':tf.Variable(tf.random_normal([128])),
        'b_fc':tf.Variable(tf.random_normal([1000])),
        'out':tf.Variable(tf.random_normal([n_classes]))
    }

    conv1 = tf.nn.relu(conv2d(x, weights['w_conv1']) + biases['b_conv1'])
    conv1 = maxpool2d(conv1)
    conv2 = tf.nn.relu(conv2d(conv1, weights['w_conv2']) + biases['b_conv2'])
    conv2 = maxpool2d(conv2)
    conv3 = tf.nn.relu(conv2d(conv2, weights['w_conv3']) + biases['b_conv3'])
    conv3 = maxpool2d(conv3)
    fc = tf.reshape(conv3,[-1, 54*54*128])
    fc = tf.matmul(fc, weights['w_fc'])+biases['b_fc']
    output = tf.matmul(fc, weights['out'])+biases['out']
    return output

In [13]:
def train_neural_network(x, epochs = 10):
    y = tf.placeholder('float', [None, n_classes])
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = prediction))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(epochs):
            # Training dataset
            accuracy_train, error_train = 0, 0
            total_batches = int(len(images_train)/BATCH_SIZE)
            for i in range(total_batches):
                image_batch, label_batch = iterator_train.get_next()
                image_batch, label_batch = sess.run([image_batch, label_batch])
                _, c = sess.run([optimizer, cost], feed_dict={
                    x: image_batch, 
                    y: label_batch
                })
                a = accuracy.eval({
                    x: image_batch, 
                    y: label_batch
                })
                error_train += c
                accuracy_train += a
                print('Epoch: ({}/{}) - Batch: ({}/{}) - Accuracy: {:.3f} - Error: {:.2f}'.format(epoch+1, epochs, i+1, total_batches, accuracy_train*100/(i+1), error_train/(i+1)))
            accuracy_train, error_train = accuracy_train/total_batches, error_train/total_batches
            # Validation dataset
            accuracy_test, error_test = 0, 0
            total_batches = int(len(images_test)/BATCH_SIZE)
            for i in range(total_batches):
                image_batch, label_batch = iterator_test.get_next()
                image_batch, label_batch = sess.run([image_batch, label_batch])
                _, c = sess.run([optimizer, cost], feed_dict={
                    x: image_batch, 
                    y: label_batch
                })
                a = accuracy.eval({
                    x: image_batch, 
                    y: label_batch
                })
                error_test += c
                accuracy_test += a
            accuracy_test, error_test = accuracy_test/total_batches, error_test/total_batches
            # Evaluation performance dataset
            print('-------------------------------------------------------------------------------------------------------------------------------')
            print('Epoch: ({}/{}) - Test error: {:.2f} - Test accuracy: {:.1f} - Train error: {:.2f} - Train accuracy: {:.1f}'.format(epoch+1, epochs, error_test, accuracy_test*100, error_train, accuracy_train*100))
            print('-------------------------------------------------------------------------------------------------------------------------------')
            
x = tf.placeholder('float', [None, 432, 432, 3])
train_neural_network(x, epochs = 10)

Instructions for updating:
Colocations handled automatically by placer.
Epoch: (1/10) - Batch: (1/26) - Accuracy: 25.000 - Error: 21991430.00
Epoch: (1/10) - Batch: (2/26) - Accuracy: 37.500 - Error: 103743083.00
Epoch: (1/10) - Batch: (3/26) - Accuracy: 31.250 - Error: 224799260.67
Epoch: (1/10) - Batch: (4/26) - Accuracy: 31.250 - Error: 250703485.50
Epoch: (1/10) - Batch: (5/26) - Accuracy: 28.750 - Error: 219902753.20
Epoch: (1/10) - Batch: (6/26) - Accuracy: 29.167 - Error: 191837161.67
Epoch: (1/10) - Batch: (7/26) - Accuracy: 30.357 - Error: 176626223.14
Epoch: (1/10) - Batch: (8/26) - Accuracy: 32.031 - Error: 167246273.25
Epoch: (1/10) - Batch: (9/26) - Accuracy: 32.639 - Error: 162889615.33
Epoch: (1/10) - Batch: (10/26) - Accuracy: 33.750 - Error: 156544501.00
Epoch: (1/10) - Batch: (11/26) - Accuracy: 34.091 - Error: 152492230.00
Epoch: (1/10) - Batch: (12/26) - Accuracy: 34.896 - Error: 147009818.83
Epoch: (1/10) - Batch: (13/26) - Accuracy: 33.654 - Error: 143240125.69
Ep

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/iterator_ops.py:408: UserWarning: An unusually high number of `Iterator.get_next()` calls was detected. This often indicates that `Iterator.get_next()` is being called inside a training loop, which will cause gradual slowdown and eventual resource exhaustion. If this is the case, restructure your code to call `next_element = iterator.get_next()` once outside the loop, and use `next_element` as the input to some computation that is invoked inside the loop.
  warnings.warn(GET_NEXT_CALL_WARNING_MESSAGE)


Epoch: (2/10) - Batch: (7/26) - Accuracy: 46.429 - Error: 6345988.07
Epoch: (2/10) - Batch: (8/26) - Accuracy: 46.875 - Error: 6699797.19
Epoch: (2/10) - Batch: (9/26) - Accuracy: 52.083 - Error: 6539541.17
Epoch: (2/10) - Batch: (10/26) - Accuracy: 51.250 - Error: 6110043.35
Epoch: (2/10) - Batch: (11/26) - Accuracy: 50.000 - Error: 6003311.32
Epoch: (2/10) - Batch: (12/26) - Accuracy: 50.521 - Error: 6095161.83
Epoch: (2/10) - Batch: (13/26) - Accuracy: 51.923 - Error: 6086179.04
Epoch: (2/10) - Batch: (14/26) - Accuracy: 54.464 - Error: 5922419.39
Epoch: (2/10) - Batch: (15/26) - Accuracy: 57.500 - Error: 5652768.64
Epoch: (2/10) - Batch: (16/26) - Accuracy: 57.812 - Error: 5368749.45
Epoch: (2/10) - Batch: (17/26) - Accuracy: 58.088 - Error: 5156372.18
Epoch: (2/10) - Batch: (18/26) - Accuracy: 59.375 - Error: 4962817.51
Epoch: (2/10) - Batch: (19/26) - Accuracy: 60.197 - Error: 4940794.33
Epoch: (2/10) - Batch: (20/26) - Accuracy: 60.938 - Error: 4994076.54
Epoch: (2/10) - Batch: 

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())